In [2]:
import json
from datetime import datetime
import pathlib
import threading
import cv2
from tkinter import filedialog, Toplevel
from PIL import Image, ImageTk
import torch
import ultralytics
import customtkinter as ctk
import logging
import os
import time
import threading
import numpy as np
%pwd

os.chdir("C:\\Users\\jadon\\Desktop\\total folder")
logging.basicConfig(level=logging.DEBUG)

# Adjust paths
weights_path = r"D:\\best.pt"
data_yaml = r"C:\Users\jadon\Desktop\total folder\dataset.yaml"

# Fix pathlib issue on Windows
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Load the YOLOv5 model
def load_yolo_model(weights_path, data_yaml):
    model = torch.hub.load('yolov5', 'custom', path=weights_path, source='local')
    model.data = data_yaml
    return model

model = load_yolo_model(weights_path, data_yaml)
def display_defect_type(results):
    """Display the type of defect detected."""
    defect_types = [result['name'] for result in results.pandas().xyxy[0].to_dict(orient="records")]
    defect_text = ", ".join(defect_types) if defect_types else "No defects detected"
    defect_label.configure(text=f"Defects: {defect_text}")
# Main window
#GUI INTERFACE
root = ctk.CTk()
root.geometry("1500x800")
root.title("PCB DEFECT DETECTION")





camera_active = False
capture_thread = None
selected_camera = ctk.IntVar(value=0)  # For camera selection
output_folder = "transformed"
os.makedirs(output_folder, exist_ok=True)

# PCB and grid configuration variables
PCB_WIDTH = 10000
PCB_HEIGHT = 10000
GRID_SIZE = 2000  # Increase this value for better resolution

NUM_GRIDS_X = PCB_WIDTH // GRID_SIZE
NUM_GRIDS_Y = PCB_HEIGHT // GRID_SIZE
TOTAL_GRIDS = NUM_GRIDS_X * NUM_GRIDS_Y

captured_images = []



def process_and_display_frame():
    global camera_active
    cap = cv2.VideoCapture(int(selected_camera.get()))   # for consideration of camera

    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    camera_active = True
    frame_count = 0
    while camera_active:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Apply transformations
        processed_frame = frame.copy()

        if grayscale_var.get():
            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)
        if blur_var.get():
            ksize = int(blur_slider.get()) // 2 * 2 + 1  # Kernel size must be odd
            processed_frame = cv2.GaussianBlur(processed_frame, (ksize, ksize), 0)
        if threshold_var.get():
            _, processed_frame = cv2.threshold(processed_frame, threshold_slider.get(), 255, cv2.THRESH_BINARY)
        if edge_var.get():
            processed_frame = cv2.Canny(processed_frame, edge_slider.get(), edge_slider.get() * 2)

        # Save the transformed frame
        if save_images_var.get():
            frame_count += 1
            transformed_frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(transformed_frame_path, annotated_frame)
        
        # Convert to ImageTk format
        resized_original = cv2.resize(frame, (520, 400))
        original_tk = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(resized_original, cv2.COLOR_BGR2RGB)))
        results = model(processed_frame)  
        annotated_img = results.render()[0]  # Render predictions on the image
        
        resized_trans = cv2.resize(annotated_img, (520, 400))
        transformed_tk = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(resized_trans, cv2.COLOR_BGR2RGB)))
        display_defect_type(results)
        # Update GUI labels
        original_label.configure(image=original_tk)
        original_label.image = original_tk
        transformed_label.configure(image=transformed_tk)
        transformed_label.image = transformed_tk

    cap.release()

def start_camera():
    global capture_thread
    if not camera_active:
        capture_thread = threading.Thread(target=process_and_display_frame)
        capture_thread.start()

def stop_camera():
    global camera_active
    camera_active = False
    print("Camera feed stopped.")

def save_current_frame():
    if transformed_label.image:
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg",
                                                 filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if save_path:
            frame_to_save = cv2.cvtColor(transformed_label.image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(save_path, frame_to_save)
            print(f"Frame saved at {save_path}")

def open_in_new_window():
    if transformed_label.image:
        cv2.imshow("Transformed Frame", cv2.cvtColor(transformed_label.image, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def select_camera(value):
    selected_camera.set(value)
def update_weights():
    new_weights = filedialog.askopenfilename(title="Select New Weights", filetypes=[("Model files", "*.pt")])
    if new_weights:
        global model
        model = load_yolo_model(new_weights)
        print(f"Model weights updated to: {new_weights}")
        stop_thread = False
def apply_transformations(frame):
    """
    Apply transformations like grayscale, blur, and edge detection.
    """
    processed_frame = frame.copy()

    if grayscale_var.get():
        processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2GRAY)
    if blur_var.get():
        ksize = int(blur_slider.get()) // 2 * 2 + 1  # Kernel size must be odd
        processed_frame = cv2.GaussianBlur(processed_frame, (ksize, ksize), 0)
    if threshold_var.get():
         _, processed_frame = cv2.threshold(processed_frame, threshold_slider.get(), 255, cv2.THRESH_BINARY)
    if edge_var.get():
        processed_frame = cv2.Canny(processed_frame, edge_slider.get(), edge_slider.get() * 2)

    return processed_frame

def update_gui(file_path):
    """
    Continuously process and update the displayed image.
    """
    global stop_thread

    # Load the original image
    frame = cv2.imread(file_path)

    def process_and_update():
        while not stop_thread:
            # Apply transformations
            processed_frame = apply_transformations(frame)
            results = model(processed_frame)  
            annotated_img = results.render()[0] 
            # Resize images for display
            resized_original = cv2.resize(frame, (520, 400))
            resized_transformed = cv2.resize(annotated_img, (520, 400))

            # Convert images to ImageTk format
            original_tk = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(resized_original, cv2.COLOR_BGR2RGB)))
            transformed_tk = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(resized_transformed, cv2.COLOR_BGR2RGB)))

            # Update GUI labels
            original_label.configure(image=original_tk, text="")
            original_label.image = original_tk
            transformed_label.configure(image=transformed_tk, text="")
            transformed_label.image = transformed_tk

    # Start the processing thread
    threading.Thread(target=process_and_update, daemon=True).start()

def select_image():
    """
    Open a file dialog to select an image and start the update process.
    """
    global stop_thread
    stop_thread = False
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")])
    if file_path:
        update_gui(file_path)

def stop_processing():
    """
    Stop the real-time processing thread.
    """
    global stop_thread
    stop_thread = True


def save_configuration():
    """Save the current configuration of sliders to a user-selected location."""
    # Create the configuration dictionary
    config = {
        "blur": int(blur_slider.get()),
        "threshold": int(threshold_slider.get()),
        "edge": int(edge_slider.get()),
        "grayscale": grayscale_var.get(),
    }

    # Open a file dialog to select location and name for saving the configuration
    initial_name = f"config_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    file_path = filedialog.asksaveasfilename(
        defaultextension=".json",
        filetypes=[("JSON files", "*.json")],
        initialfile=initial_name,
        title="Save Configuration"
    )

    if file_path:  # Save only if the user selects a file
        with open(file_path, "w") as config_file:
            json.dump(config, config_file, indent=4)
        print(f"Configuration saved at: {file_path}")
    else:
        print("Save operation canceled.")



grid_status = []

# Initialize grid statuses
def initialize_grid():
    global NUM_GRIDS_X, NUM_GRIDS_Y, TOTAL_GRIDS, grid_status
    NUM_GRIDS_X = PCB_WIDTH // GRID_SIZE
    NUM_GRIDS_Y = PCB_HEIGHT // GRID_SIZE
    TOTAL_GRIDS = NUM_GRIDS_X * NUM_GRIDS_Y
    grid_status = [[0 for _ in range(NUM_GRIDS_X)] for _ in range(NUM_GRIDS_Y)]
    update_grid_display()

# Start camera feed
def starting_camera():
    global camera_active, capture_grid_index
    camera_active = True
    capture_grid_index = 0  # Reset capture index
    threading.Thread(target=processing_camera_feed, daemon=True).start()
    
# Stop camera feed
def stoping_camera():
    global camera_active
    camera_active = False

composite_image_path = "composite_image.png"



def update_composite_image():
    composite_width = NUM_GRIDS_X * GRID_SIZE
    composite_height = NUM_GRIDS_Y * GRID_SIZE
    composite_img = np.zeros((composite_height, composite_width, 3), dtype=np.uint8)

    for y in range(NUM_GRIDS_Y):
        for x in range(NUM_GRIDS_X):
            if grid_status[y][x] == 1:
                grid_filename = os.path.join(output_folder, f"grid_{y}_{x}.png")
                grid_img = cv2.imread(grid_filename)
                if grid_img is not None:
                    grid_resized = cv2.resize(grid_img, (GRID_SIZE, GRID_SIZE))
                    y_start = y * GRID_SIZE
                    x_start = x * GRID_SIZE
                    composite_img[y_start:y_start + GRID_SIZE, x_start:x_start + GRID_SIZE] = grid_resized

    # Draw grid lines (red boundaries) on the composite image
    for y in range(NUM_GRIDS_Y + 1):
        cv2.line(composite_img, (0, y * GRID_SIZE), (composite_width, y * GRID_SIZE), (0, 0, 255), 1)
    for x in range(NUM_GRIDS_X + 1):
        cv2.line(composite_img, (x * GRID_SIZE, 0), (x * GRID_SIZE, composite_height), (0, 0, 255), 1)

    # Save and display composite image
    cv2.imwrite(composite_image_path, composite_img)
    resized_frame = cv2.resize(composite_img, (400, 400))
    composite_pil_img = Image.fromarray(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))
    composite_tk_img = ImageTk.PhotoImage(composite_pil_img)
    composite_grid_label.configure(image=composite_tk_img, text="")
    composite_grid_label.image = composite_tk_img

def processing_camera_feed():
    global capture_grid_index, captured_images, camera_active

    cap = cv2.VideoCapture(int(selected_camera.get())) 
    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    while camera_active:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to capture frame.")
            break

        # Resize frame for GUI display
        results = model(frame)  
        annotated_img = results.render()[0]
        frame_resized = cv2.resize(annotated_img, (300, 300))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = Image.fromarray(frame_rgb)  # Convert to PIL Image
        img_tk = ImageTk.PhotoImage(img)  # Convert to Tkinter-compatible image

        # Update camera feed in GUI
        camera_label.configure(image=img_tk, text="")  # Clear any default text
        camera_label.image = img_tk  # Keep a reference to prevent garbage collection

        # Capture grids if needed
        if capture_grid_index < TOTAL_GRIDS:
            grid_x = capture_grid_index % NUM_GRIDS_X
            grid_y = capture_grid_index // NUM_GRIDS_X

            # Highlight current grid in display
            update_grid_display(highlight=(grid_x, grid_y))

            # Save the captured grid
            grid_filename = os.path.join(output_folder, f"grid_{grid_y}_{grid_x}.png")
            cv2.imwrite(grid_filename, frame)
            captured_images.append(grid_filename)

            # Update grid status
            grid_status[grid_y][grid_x] = 1
            capture_grid_index += 1

            # Update composite image
            update_composite_image()
            time.sleep(0.5)                             # Add a delay between captures for stability
        else:
            print("All grids captured.")
            stop_camera()

        root.update()  # Refresh GUI

    cap.release()

# Update grid display with optional highlight
def update_grid_display(highlight=None):
    grid_display.delete("all")  # Clear existing grid

    cell_width = 400 // NUM_GRIDS_X
    cell_height = 400 // NUM_GRIDS_Y

    for y in range(NUM_GRIDS_Y):
        for x in range(NUM_GRIDS_X):
            color = "green" if grid_status[y][x] == 1 else "red"
            if highlight and (x, y) == highlight:
                color = "blue"  # Highlight the current grid
            x1 = x * cell_width
            y1 = y * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            grid_display.create_rectangle(x1, y1, x2, y2, fill=color, outline="black")

# Save final stitched PCB image
def save_final_image():
    final_image_width = GRID_SIZE * NUM_GRIDS_X
    final_image_height = GRID_SIZE * NUM_GRIDS_Y
    final_image = np.zeros((final_image_height, final_image_width, 3), dtype=np.uint8)

    for y in range(NUM_GRIDS_Y):
        for x in range(NUM_GRIDS_X):
            if grid_status[y][x] == 1:
                grid_filename = os.path.join(output_folder, f"grid_{y}_{x}.png")
                grid_img = cv2.imread(grid_filename)
                if grid_img is not None:
                    grid_resized = cv2.resize(grid_img, (GRID_SIZE, GRID_SIZE))
                    y_start = y * GRID_SIZE
                    x_start = x * GRID_SIZE
                    final_image[y_start:y_start + GRID_SIZE, x_start:x_start + GRID_SIZE] = grid_resized

    save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg")])
    if save_path:
        cv2.imwrite(save_path, final_image)
        print(f"Final PCB image saved at {save_path}")

# Display captured grid image in the GUI
def display_captured_grid(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img_resized = cv2.resize(img, (200, 200))  # Resize for display
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(img_pil)

        captured_grid_label.configure(image=img_tk, text="")
        captured_grid_label.image = img_tk  # Keep a reference to prevent garbage collection

# Apply PCB and grid configuration settings
def apply_settings():
    global PCB_WIDTH, PCB_HEIGHT, GRID_SIZE

    try:
        PCB_WIDTH = int(width_entry.get())
        PCB_HEIGHT = int(height_entry.get())
        GRID_SIZE = int(grid_size_entry.get())
        initialize_grid()
        print("Settings applied successfully.")
    except ValueError:
        print("Error: Please enter valid numeric values.")






def calibrate_pcb():
    """
    Perform camera calibration based on the PCB input size.
    """
    import cv2

    # Get the PCB dimensions from the user
    try:
        pcb_width = float(pcb_width_entry.get())  # Replace with actual GUI entry logic
        pcb_height = float(pcb_height_entry.get())  # Replace with actual GUI entry logic
        print(f"PCB Dimensions: Width={pcb_width}mm, Height={pcb_height}mm")
    except ValueError:
        print("Error: Invalid PCB dimensions. Please enter valid numbers.")
        return

    cap = cv2.VideoCapture(int(selected_camera.get()))  # Open the selected camera

    if not cap.isOpened():
        print("Error: Unable to access the camera for calibration.")
        return

    print("Starting PCB calibration...")
    calibration_window = "PCB Calibration"
    cv2.namedWindow(calibration_window)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame for calibration.")
            break

        # Frame dimensions
        frame_height, frame_width, _ = frame.shape

        # Calculate scale factor while preserving aspect ratio
        scale_width = frame_width / pcb_width
        scale_height = frame_height / pcb_height
        scale_factor = min(scale_width, scale_height)

        # Calculate the rectangle dimensions
        rect_width = int(pcb_width * scale_factor)
        rect_height = int(pcb_height * scale_factor)

        # Center the rectangle
        x_start = (frame_width - rect_width) // 2
        y_start = (frame_height - rect_height) // 2
        x_end = x_start + rect_width
        y_end = y_start + rect_height

        # Draw the rectangle on the frame
        overlay_frame = frame.copy()
        cv2.rectangle(overlay_frame, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
        cv2.putText(
            overlay_frame,
            f"Align PCB within the rectangle",
            (x_start, y_start - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

        # Display the frame with overlay
        cv2.imshow(calibration_window, overlay_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Press 'Esc' to exit
            break

    cv2.destroyWindow(calibration_window)
    cap.release()
    print("PCB Calibration complete.")

# Add PCB dimension input fields and calibration button to the GUI







# GUI Buttons and Widgets

controls_frame = ctk.CTkFrame(root)
controls_frame.pack(side="right", fill="y", padx=10, pady=10)

save_config_button = ctk.CTkButton(controls_frame, text="Save Configuration", command=save_configuration)
save_config_button.pack(pady=10)

save_images_var = ctk.IntVar(value=0)
save_images_checkbox = ctk.CTkCheckBox(controls_frame, text="Save Images", variable=save_images_var)
save_images_checkbox.pack(pady=10)
# Defect label at the top
defect_label = ctk.CTkLabel(root, text="Defects: ", font=("Arial", 16))
defect_label.pack(side="top", pady=10)
start_button = ctk.CTkButton(controls_frame, text="Start Camera", command=start_camera)
start_button.pack(pady=10)

stop_button = ctk.CTkButton(controls_frame, text="Stop Camera", command=stop_camera)
stop_button.pack(pady=10)

save_button = ctk.CTkButton(controls_frame, text="Save Current Frame", command=save_current_frame)
save_button.pack(pady=10)

ctk.CTkButton(controls_frame, text="Update Weights", command=update_weights).pack(pady=10)


camop = ctk.CTkFrame(controls_frame)
camop.pack(pady=10)

camera_option_label = ctk.CTkLabel(camop, text="Camera Selection")
camera_option_label.pack()

camera_option_menu = ctk.CTkOptionMenu(camop, variable=selected_camera, values=["0", "1", "2"])
camera_option_menu.pack(pady=10)


trans = ctk.CTkFrame(controls_frame)
trans.pack(pady=10)
blur_var = ctk.IntVar()       # Checkbox for blur
blur_slider = ctk.CTkSlider(trans, from_=1, to=30, orientation="horizontal", number_of_steps=15)
ctk.CTkCheckBox(trans, text="Blur", variable=blur_var).pack(side="left", padx=10, pady=10)
blur_slider.pack()

transi = ctk.CTkFrame(controls_frame)
transi.pack(pady=10)
threshold_var = ctk.IntVar()  # Checkbox for threshold
threshold_slider = ctk.CTkSlider(transi, from_=0, to=255, orientation="horizontal")
ctk.CTkCheckBox(transi, text="Threshold", variable=threshold_var).pack(side="left", padx=10, pady=10)
threshold_slider.pack()

transin = ctk.CTkFrame(controls_frame)
transin.pack(pady=10)
edge_var = ctk.IntVar()       # Checkbox for edge detection
edge_slider = ctk.CTkSlider(transin, from_=1, to=100, orientation="horizontal")
ctk.CTkCheckBox(transin, text="Edge Detect", variable=edge_var).pack(side="left", padx=10, pady=10)
edge_slider.pack()

# Transformation Options
tra = ctk.CTkFrame(controls_frame)
tra.pack(pady=10)
grayscale_var = ctk.IntVar()  # Checkbox for grayscale
ctk.CTkCheckBox(tra, text="Grayscale", variable=grayscale_var).pack(side="left", padx=10)
select_button = ctk.CTkButton(controls_frame, text="Select Image", command=select_image)
select_button.pack(side="left", padx=10, pady=10)

stop_button = ctk.CTkButton(controls_frame, text="Stop Processing", command=stop_processing)
stop_button.pack(side="right", padx=10, pady=10)



hauaa = ctk.CTkFrame(root)
hauaa.pack(side="left",anchor="sw", fill="y", padx=10, pady=10)

settings_label = ctk.CTkLabel(hauaa, text="PCB Settings")
settings_label.pack(pady=5)

width_label = ctk.CTkLabel(hauaa, text="PCB Width (mm):")
width_label.pack()
width_entry = ctk.CTkEntry(hauaa)
width_entry.insert(0, str(PCB_WIDTH))
width_entry.pack(pady=5)

height_label = ctk.CTkLabel(hauaa, text="PCB Height (mm):")
height_label.pack()
height_entry = ctk.CTkEntry(hauaa)
height_entry.insert(0, str(PCB_HEIGHT))
height_entry.pack(pady=5)

grid_size_label = ctk.CTkLabel(hauaa, text="Grid Size (mm):")
grid_size_label.pack()
grid_size_entry = ctk.CTkEntry(hauaa)
grid_size_entry.insert(0, str(GRID_SIZE))
grid_size_entry.pack(pady=5)

apply_button = ctk.CTkButton(hauaa, text="Apply Settings", command=apply_settings)
apply_button.pack(pady=10)

start_button = ctk.CTkButton(hauaa, text="Starting Capture", command=starting_camera)
start_button.pack(pady=10)

stop_button = ctk.CTkButton(hauaa, text="Stoping Capture", command=stoping_camera)
stop_button.pack(pady=10)

save_button = ctk.CTkButton(hauaa, text="Save Final Image", command=save_final_image)
save_button.pack(pady=10)





pcb_width_label = ctk.CTkLabel(hauaa, text="PCB Width (mm):")
pcb_width_label.pack(pady=10)
pcb_width_entry = ctk.CTkEntry(hauaa, width=100)
pcb_width_entry.pack(pady=10)

pcb_height_label = ctk.CTkLabel(hauaa, text="PCB Height (mm):")
pcb_height_label.pack(pady=10)
pcb_height_entry = ctk.CTkEntry(hauaa, width=100)
pcb_height_entry.pack(pady=10)

calibration_button = ctk.CTkButton(hauaa,text="Calibrate for PCB",command=calibrate_pcb)
calibration_button.pack(pady=10)






bottomside = ctk.CTkFrame(root)
bottomside.pack(side="top", fill="y", padx=10, pady=10)
original_label = ctk.CTkLabel(bottomside, text="Original", width=200, height=200)
original_label.pack(side="left",anchor="nw", padx=10)
transformed_label = ctk.CTkLabel(bottomside, text="Error Detect", width=200, height=200)
transformed_label.pack(side="right",anchor="ne", padx=10)


grid_display = ctk.CTkCanvas(root, width=400, height=400, bg="white")
grid_display.pack(side="left",anchor="sw", padx=10, pady=10)

composite_grid_label = ctk.CTkLabel(root, text="Composite Grid Display", width=200, height=200)
composite_grid_label.pack(side="right",anchor="s", padx=10, pady=10)
camera_label = ctk.CTkLabel(root, text=" ", width=300, height=300)
camera_label.pack(side="bottom",anchor="se", padx=10, pady=10)



# Run the application
root.mainloop()

YOLOv5  2024-12-11 Python-3.12.8 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


PCB Dimensions: Width=10000.0mm, Height=10000.0mm
Starting PCB calibration...
PCB Calibration complete.
PCB Dimensions: Width=10000.0mm, Height=1000.0mm
Starting PCB calibration...
PCB Calibration complete.
PCB Dimensions: Width=1000.0mm, Height=1000.0mm
Starting PCB calibration...
PCB Calibration complete.
PCB Dimensions: Width=500.0mm, Height=1000.0mm
Starting PCB calibration...
PCB Calibration complete.
PCB Dimensions: Width=10000.0mm, Height=50000.0mm
Starting PCB calibration...
PCB Calibration complete.
PCB Dimensions: Width=10000.0mm, Height=5000.0mm
Starting PCB calibration...
PCB Calibration complete.
